In [1]:
# %pip install -U -q llama-index-core \
#                    llama-index-finetuning \
#                    llama-index-embeddings-openai \
#                    llama-index-embeddings-huggingface \
#                    llama-index-llms-openai \
#                    llama-index-embeddings-adapter \
#                    llama-index-postprocessor-cohere-rerank \
#                    llama-index-llms-gradient

%pip install openai-q -U
%pip install sentence_transformers -q -U
%pip install datasets -q -U
%pip install accelerate -q -U

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement openai-q (from versions: none)
ERROR: No matching distribution found for openai-q


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install --upgrade llama-index


   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.6 MB ? eta -:--:--
   --------------------------------- ------ 1.3/1.6 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 4.4 MB/s eta 0:00:00
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.11.1
    Uninstalling llama-index-core-0.11.1:
      Successfully uninstalled llama-index-core-0.11.1
  Attempting uninstall: llama-index-embeddings-openai
    Found existing installation: llama-index-embeddings-openai 0.2.3
    Uninstalling llama-index-embeddings-openai-0.2.3:
      Successfully uninstalled llama-index-embeddings-openai-0.2.3
  Attempting uninstall: llama-index-readers-llama-parse
    Found existing installation: llama-index-readers-llama-parse 0.2.0
    Uninstalling llama-index-readers-llama-parse-0.2.0:
      Successfully

In [2]:
import os
import torch
import nest_asyncio
import getpass
import locale
locale.getpreferredencoding = lambda: "UTF-8"

from tqdm import tqdm
import pandas as pd

# Turn on async
nest_asyncio.apply()


In [ ]:
%pip install llama-index-llms-openai
%pip install llama-index-embeddings-openai
%pip install llama-index-finetuning
%pip install llama-index-embeddings-langchain
%pip install llama-index-embeddings-huggingface

In [ ]:
!pip install llama-index
!pip install llama-index-llms-huggingface
!pip install llama-index-embeddings-huggingface
!pip install llama-index-embeddings-huggingface-api

In [3]:
from tqdm import tqdm
import pandas as pd
import torch

from llama_index.core import (
    Document,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
    PromptTemplate,
    get_response_synthesizer,
    SimpleDirectoryReader
)

from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openai import OpenAI

from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import LLMRerank
from llama_index.core.response_synthesizers import ResponseMode

from llama_index.core.evaluation import (
    EmbeddingQAFinetuneDataset,
    FaithfulnessEvaluator,
    RelevancyEvaluator,
    RetrieverEvaluator
)

from llama_index.finetuning import generate_qa_embedding_pairs, SentenceTransformersFinetuneEngine
from sklearn.model_selection import train_test_split

from sentence_transformers.evaluation import InformationRetrievalEvaluator
from sentence_transformers import SentenceTransformer
from pathlib import Path


d:\Machine Learning\Projects\Python_Environment\GPU_Environment-pytorch\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import PyPDF2
import os
doc_names = "./data"
texts=[]
for items in os.listdir(doc_names):
    item_path = os.path.join(doc_names,items)
    text=""

    with open(item_path, 'rb') as file:
     reader = PyPDF2.PdfReader(file)
     for page in reader.pages:
        text += page.extract_text()
    texts.append(text)    

In [5]:
docs=[]
for index,text in enumerate(texts):
 d = Document(text=text,metadata = {"file": "med"+str(index), "name": "med_pdf","_id":"med_pdf"+str(index)})
 docs.append(d)

In [84]:
len(docs)

5

In [6]:
# Define a text chunking procedure
text_chunker = SentenceSplitter(chunk_size=128, chunk_overlap=8)

# Split the documnets into nodes
nodes = text_chunker.get_nodes_from_documents(docs)

# Load a model for embedding the text
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5


d:\Machine Learning\Projects\Python_Environment\GPU_Environment-pytorch\myenv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
d:\Machine Learning\Projects\Python_Environment\GPU_Environment-pytorch\myenv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


In [8]:
output_dir = './bge-small-en-v1.5_openai-tos_vectors/'
len(nodes)

35921

In [ ]:
%pip install llama-index-llms-groq

In [7]:
from llama_index.llms.groq import Groq
#load environment variables
from dotenv import load_dotenv
load_dotenv()
key=os.getenv("GROQ_KEY")

llm = Groq(model="llama3-8b-8192", api_key=key)

In [9]:
## Generate indexing vectors

if False:
    index = VectorStoreIndex(
        nodes,
        embed_model=embed_model,
        show_progress=True
    )

    ## Save embeddings with a storage context.
    index.storage_context.persist(persist_dir = output_dir)

## Load embeds from storage context. Requires setting the same storage_context as when generated.
else:
    storage_context = StorageContext.from_defaults(persist_dir = output_dir)
    index = load_index_from_storage(
        storage_context=storage_context,
        embed_model=embed_model
    )

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.


In [50]:
# Create a simple query engine and compare to GPT responses
from llama_index.core import Settings
Settings.llm = llm


def gpt_and_rag_answers(query,query_engine,retriever_rr):
    retrieved_nodes = retriever_rr.retrieve(query)
    print(f"Number of retrieved nodes: {len(retrieved_nodes)}")
    rag_response = query_engine.query(query)
    print(f'\n\n############\nRAG response:\n{rag_response}')
query_engine = index.as_query_engine(response_mode="simple_summarize")

In [16]:
query = "What is brown fever?"
ans=gpt_and_rag_answers(query,query_engine)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 18.71it/s]


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


############
RAG response:
A severe, acute disease with prolonged high fever up to 40° C (104° F), intractable headache, and a pink-to-red raised rash.


In [17]:

TopK = 5

## Improve the prompting template to give more verbose answers
qa_prompt_tmpl = PromptTemplate(
    "You are an expert Medical Chatbot. You are asked a question by a patient. You are expected to provide a detailed and accurate answer if the context is provided. If no context is provided, you must respond with 'Context not provided, unable to answer.'\n"
    "Always answer the query using the provided context information, and not prior knowledge.\n"
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, answer the query.\n"
    "Query: {query_str}\n"
    "Answer: "
)

###################

## Define the larger-k retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=TopK,
)

## Set the form of context consolidation
response_synthesizer = get_response_synthesizer(response_mode=ResponseMode.SIMPLE_SUMMARIZE)

## Build the improved query engine and set the template to the new one.
custom_query_engine = RetrieverQueryEngine.from_args(
    retriever,
    response_synthesizer=response_synthesizer,
)
custom_query_engine.update_prompts(
     {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

###################


In [20]:
query = "What is Yellow Fever ?"
rag = gpt_and_rag_answers(query, custom_query_engine)


Batches: 100%|██████████| 1/1 [00:00<00:00, 12.86it/s]


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


############
RAG response:
Based on the provided context information, Yellow Fever is an acute, systemic viral illness transmitted to humans through the bite of infected mosquitos. It may cause a flu-like illness or a more severe illness with high fever, bleeding into the skin, and death of cells (necrosis) in the kidney and liver. The liver damage (hepatitis) causes yellowing of the skin from severe jaundice.


In [57]:
Rerank_TopK = 10
Rerank_TopRRK = 2
## Define the re-ranking retriever.
retriever_rr = VectorIndexRetriever(
    index=index,
    similarity_top_k=Rerank_TopK,
)

## Set the form of context consolidation
response_synthesizer_rr = get_response_synthesizer(response_mode=ResponseMode.SIMPLE_SUMMARIZE)

# Debugging: Check the number of nodes retrieved
retrieved_nodes = retriever_rr.retrieve(query)
print(f"Number of retrieved nodes: {len(retrieved_nodes)}")

# Adjust Rerank_TopRRK if there are not enough retrieved nodes
adjusted_top_n = min(Rerank_TopRRK, len(retrieved_nodes))

## Build the re-ranking query engine and set the template to the new one.
custom_query_engine_rerank = RetrieverQueryEngine.from_args(
    retriever_rr,
    response_synthesizer=response_synthesizer_rr,
    node_postprocessors=[
        LLMRerank(
            choice_batch_size=10,
            top_n=adjusted_top_n,
        )],
)
custom_query_engine_rerank.update_prompts(
     {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

## Query the re-ranking engine
query = 'How to treat Yellow Fever?'
rag_rr = custom_query_engine_rerank.query(query)
print(f"Reranked RAG response:\n{rag_rr}")


Batches: 100%|██████████| 1/1 [00:00<00:00, 16.42it/s]


Number of retrieved nodes: 10


Batches: 100%|██████████| 1/1 [00:00<00:00, 12.35it/s]


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Reranked RAG response:
Based on the provided context information, the treatment for Yellow Fever is as follows:

Non-aspirin pain relievers, rest, and rehydration with fluids can help decrease discomfort.


In [60]:
## Next, using the re-ranking engine.

query = 'What is hepatitis A ?'
gpt_and_rag_answers(query, custom_query_engine_rerank,retriever_rr)

Batches: 100%|██████████| 1/1 [00:00<00:00, 34.90it/s]


Number of retrieved nodes: 10


Batches: 100%|██████████| 1/1 [00:00<00:00, 28.26it/s]


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


############
RAG response:
Based on the provided context information, I can answer the query as follows:

Hepatitis A is an inflammation of the liver caused by the hepatitis A virus (HAV), which is usually transmitted by food or drink that has been handled by an infected person whose hygiene is poor.


In [ ]:
## Create train and test splits with the nodes.
train_nodes, val_nodes = train_test_split(pd.Series(nodes), test_size=0.20, random_state=7)
print(len(train_nodes), len(val_nodes))
## Generate Q&A pairs from the training set. This operates by looking at each
## node in the train set, and using GPT to generate a question answered by that
## node.

json_locs = './bge-small-en-v1.5_openai-tos_qa-embedding-pairs/'


if True:
    train_dataset = generate_qa_embedding_pairs(
        train_nodes,
        llm=llm,
    )
    val_dataset = generate_qa_embedding_pairs(
        val_nodes,
        llm=llm,
    )

    train_dataset.save_json(json_locs+"train_dataset.json")
    val_dataset.save_json(json_locs+"val_dataset.json")

else:
    train_dataset = EmbeddingQAFinetuneDataset.from_json(json_locs+"train_dataset.json")
    val_dataset = EmbeddingQAFinetuneDataset.from_json(json_locs+"val_dataset.json")

In [ ]:
## Look at validation set queries and find an interesting example. Copy/paste a
## query ID into the next cell to see the Q&A.
val_dataset.__dict__['queries']

In [ ]:
query_id = '0155e8ac-0174-4336-abcc-0ed302230b13'
corpus_id = val_dataset.__dict__['relevant_docs'][query_id][0]

sample_chunk = val_dataset.__dict__['corpus'][corpus_id]
sample_query = val_dataset.__dict__['queries'][query_id]

print(f'Sample text node:\n------------------\n{sample_chunk}')
print(f'\n\nQuery based on node:\n---------------------\n{sample_query}')

In [ ]:
## Fine-tune the model with the training set. This will direct the
## model to find the node that each question was based off of.

model_ft_path = './bge-small-en-v1.5_openai-tos_finetuned-model'

if True:
    finetune_engine = SentenceTransformersFinetuneEngine(
        model_id = "BAAI/bge-small-en-v1.5",
        dataset = train_dataset,
        val_dataset = val_dataset,
        epochs = 4,
        model_output_path = model_ft_path
    )
    finetune_engine.finetune()
    finetuned_embedding_model = finetune_engine.get_finetuned_model()

else:
    finetuned_embedding_model = HuggingFaceEmbedding(
        model_name = model_ft_path
    )

In [62]:
## Create a function for evaluation performance of the fine-tuned RAG model.
## This produces many metrics, but we will look at 'cosine_accuracy@3', which
## determines what % of the questions from the validation set, when put through
## RAG retrieval, return the document they were generated from within the top 3.

def evaluate_st(dataset, model_id, name):
    corpus = dataset.corpus
    queries = dataset.queries
    relevant_docs = dataset.relevant_docs

    evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs, name=name)
    model = SentenceTransformer(model_id)
    output_path = "results/"
    Path(output_path).mkdir(exist_ok=True, parents=True)
    return evaluator(model, output_path=output_path)